In [3]:
import os
import pandas as pd
import numpy as np
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.utils import to_time_series_dataset

# DTW距離を算出
def dtw(x, y):
    # xのデータ数，yのデータ数をそれぞれTx,Tyに代入
    Tx = len(x)
    Ty = len(y)
    
    # C:各マスの累積コスト，　B：最小コストの行/列番号
    C = np.zeros((Tx, Ty))
    B = np.zeros((Tx, Ty, 2), int)
    
    # 一番初めのマスのコストを，xとyのそれぞれ一番初めの値にする
    C[0, 0] = dist(x[0], y[0])
    
    # 動的計画法を用いる
    # 左下のマスからスタートし，各マスに到達するため最小の累積コストを1マスずつ求める
    
    # 境界条件：両端が左下と右上にあること
    # 単調性：左下から始まり，右，上，右上のいずれかにしか進まないこと
    # 連続性：繋がっていること
    
    # 一番下の行は，真っ直ぐ右にコストが累積される
    for i in range(Tx):
        C[i, 0] = C[i - 1, 0] + dist(x[i], y[0])
        B[i, 0] = [i - 1, 0]
        
    # 同様に一番左の列は，真っ直ぐ上にコストが累積される
    for j in range(1, Ty):
        C[0, j] = C[0, j - 1] + dist(x[0], y[j])
        B[0, j] = [0, j - 1]
        
    # その他のマスの累積コストを求める
    for i in range(1, Tx):
        for j in range(1, Ty):
            pi, pj, m = get_min(C[i - 1, j],
                                C[i, j - 1],
                                C[i - 1, j - 1],
                                i, j)
            # get_minで返ってきた最小コストを累積コストに足す
            C[i, j] = dist(x[i], y[j]) + m
            # get_minで返ってきた最小コストの行/列番号を保持
            B[i, j] = [pi, pj]
    # 最終的な右上（最終の到達点）のコスト
    cost = C[-1, -1]
    
    path = [[Tx - 1, Ty - 1]]
    
    # 逆順にたどることでパスを求める
    i = Tx - 1
    j = Ty - 1
    
    while((B[i, j][0] != 0) or (B[i, j][1] != 0)):
        path.append(B[i, j])
        i, j = B[i, j].astype(int)
    path.append([0, 0])
    return np.array(path), cost, C

# 距離算出
def dist(a, b):
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

# 最小値算出
def get_min(m0, m1, m2, i, j):
    if m0 < m1:
        if m0 < m2:
            return i - 1, j, m0
        else:
            return i - 1, j - 1, m2
    else:
        if m1 < m2:
            return i, j - 1, m1
        else:
            return i - 1, j - 1, m2

In [16]:
data_b1 = pd.read_csv("/Users/yuki-f/scratchsearch/splitted/11242.csv", encoding="shift-jis", usecols=['x','y']).values
data_b2 = pd.read_csv("/Users/yuki-f/scratchsearch/splitted/12404.csv", encoding="shift-jis", usecols=['x','y']).values

In [17]:
data_b1 = TimeSeriesScalerMinMax().fit_transform(to_time_series_dataset([data_b1])).flatten().reshape(-1, 2)
data_b2 = TimeSeriesScalerMinMax().fit_transform(to_time_series_dataset([data_b2])).flatten().reshape(-1, 2)

In [18]:
dtw(data_b1, data_b2)[1]

1.551204877308251

In [22]:
data_c3 = pd.read_csv("/Users/yuki-f/scratchsearch/splitted/6304.csv", encoding="shift-jis", usecols=['x','y']).values
data_c4 = pd.read_csv("/Users/yuki-f/scratchsearch/splitted/8296.csv", encoding="shift-jis", usecols=['x','y']).values

In [23]:
data_c3 = TimeSeriesScalerMinMax().fit_transform(to_time_series_dataset([data_c3])).flatten().reshape(-1, 2)
data_c4 = TimeSeriesScalerMinMax().fit_transform(to_time_series_dataset([data_c4])).flatten().reshape(-1, 2)

In [24]:
dtw(data_c3, data_c4)[1]

3.802633338265561